不带硬算子的socket脚本

首先import相关的包，分为三部分， 框架下相关包，icraft相关包，模型的前后处理、可视化
* xir:Icraft-XIR的python模块
* xrt:Icraft-XIR的python模块(runtime运行时)
* host_backend:Icraft-HostBackend的python模块
* buyibackend:Icraft-BuyiBackend的python模块
* preprocess:前处理模块
* detpost_soft:不配硬算子的后处理模块
* visualize:结果可视化模块

In [12]:
import numpy as np
import cv2

from icraft.xir import *
from icraft.xrt import *
from icraft.host_backend import *
from icraft.buyibackend import *

from preprocess import letterbox,scale_coords
from detpost_soft import get_det_results,non_max_suppression
from visualize import vis,COCO_CLASSES

声明一些文件路径，主要是板子ip地址、icraft编译后的指令生成网络（json&raw文件）、测试图片

In [13]:
# ---------------------------------参数设置---------------------------------
# 路径设置 
URL_PATH =  R"socket://ql100aiu@192.168.125.184:9981?npu=0x40000000&dma=0x80000000"
GENERATED_JSON_FILE = "./imodel/yolov5s_soft/yolov5s_BY.json"
GENERATED_RAW_FILE = "./imodel/yolov5s_soft/yolov5s_BY.raw"
IMG_PATH="./images/000000000885.jpg" 

下面是最主要的模型前向过程，分为以下几个步骤   
* 1.开启设备
* 2.加载测试图像并转成icraft.Tensor
* 3.加载指令生成后的网络
* 4.创建Session，将整个网络绑定到BuyiBackend和HostBackend，使用Buyi_device和HostDevice
* 5.进行模型前向推理

In [14]:
# 开启设备
device = Device.Open(URL_PATH)
# # 关闭mmu模式
# device.cast<BuyiDevice>().mmuModeSwitch(False);

In [15]:
# 加载测试图片
img=cv2.imread(IMG_PATH)
ri = letterbox(img,new_shape=(640,640),stride=32,auto=False)[0] #ndarray of shape [640,640,3]
img_=ri.astype(np.float32).reshape(1,640,640,3)
input_tensor = Tensor(img_, Layout("NHWC"))
print('INFO: load test image!')

INFO: load test image!


In [16]:
# 加载指令生成后的网络
generated_network = Network.CreateFromJsonFile(GENERATED_JSON_FILE)
generated_network.loadParamsFromFile(GENERATED_RAW_FILE)
print('INFO: Create network!')

INFO: Create network!


In [17]:
# 创建Session
session = Session.Create([ BuyiBackend, HostBackend], generated_network.view(0), [ device,HostDevice.Default()])
session.apply()
# 模型前向推理
generated_output = session.forward( [input_tensor] )
# 养成良好的使用习惯，网络前向结束后，重置device 
device.reset(1)

此时，我们就获得了模型前向推理的结果，可以看一下三个检测头的输出结果！

In [18]:
print('anchor_0 shape =',np.array(generated_output[0]).shape) # [1,80,80,255]
print('anchor_1 shape =',np.array(generated_output[1]).shape) # [1,40,40,255]
print('anchor_2 shape =',np.array(generated_output[2]).shape) # [1,20,20,255]
print('INFO: get forward results!')

anchor_0 shape = (1, 80, 80, 255)
anchor_1 shape = (1, 40, 40, 255)
anchor_2 shape = (1, 20, 20, 255)
INFO: get forward results!


接下来对输出结果进行后处理，主要进行输出数据的排布转换、NMS非极大值抑制抑制，分别对应get_det_results和non_max_suppression，有兴趣的可自行查看封装好的函数内部实现过程。

In [19]:
# ---------------------------------后处理---------------------------------
# from socket results to det results 
pred = get_det_results(generated_output)
print("INFO: pred shape =",pred.shape)
# NMS
pred = non_max_suppression(pred,classes=None, agnostic=False, max_det=1000)

INFO: pred shape = torch.Size([1, 25200, 85])


最终，将结果进行可视化，就可以看到指令生成后的上板运行结果啦~是不是很简便，快来尝试着部署自己的网络吧:)

In [20]:
# ---------------------------------结果可视化---------------------------------
det = pred[0]
det[:, :4] = scale_coords(img_.shape[1:], det[:, :4], img.shape).round()
result_image = vis(img, boxes=det[:,:4], scores=det[:,4], cls_ids=det[:,5], conf=0.25, class_names=COCO_CLASSES)
cv2.imshow(" ", result_image)
cv2.waitKey(0)
cv2.destroyAllWindows()

如果需要继续测试其他图片，只需要加载新的测试图片、重置device、重新调用session.forward即可。
建议养成良好的使用习惯，每次session.forward后重置device，确保连续测试不同图像的正确性。

In [21]:
IMG1_PATH="./images/000000000139.jpg" 
# 加载新的测试图片
img=cv2.imread(IMG1_PATH)
ri = letterbox(img,new_shape=(640,640),stride=32,auto=False)[0] #ndarray of shape [640,640,3]
img_=ri.astype(np.float32).reshape(1,640,640,3)
input_tensor = Tensor(img_, Layout("NHWC"))
print('INFO: load test image!')
# 模型前向推理
generated_output = session.forward( [input_tensor] )
# 重置device   这一步很重要！！！重置device   这一步很重要！！！重置device   这一步很重要！！！
device.reset(1)
# ---------------------------------后处理---------------------------------
# from socket results to det results 
pred = get_det_results(generated_output)
print("INFO: pred shape =",pred.shape)
# NMS
pred = non_max_suppression(pred,classes=None, agnostic=False, max_det=1000)
# ---------------------------------结果可视化---------------------------------
det = pred[0]
det[:, :4] = scale_coords(img_.shape[1:], det[:, :4], img.shape).round()
result_image = vis(img, boxes=det[:,:4], scores=det[:,4], cls_ids=det[:,5], conf=0.25, class_names=COCO_CLASSES)
cv2.imshow(" ", result_image)
cv2.waitKey(0)
cv2.destroyAllWindows()

INFO: load test image!
INFO: pred shape = torch.Size([1, 25200, 85])


In [22]:
# 关闭设备
Device.Close(device)